In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
import json
import csv

import requests
import time

from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from snips_nlu import SnipsNLUEngine
from snips_nlu.default_configs import CONFIG_EN

from numpy import exp
import collections

In [ ]:
# constants
NLU_NAME = 'luis'
VERSION = 6

# paths
TRAIN_FILE = '../datasets/' + NLU_NAME + '_train_v' + str(VERSION) + '.json'
TEST_FILE = '../datasets/' + NLU_NAME + '_test_v' + str(VERSION) + '.csv'
OUTPUT_FILE = '../results/' + NLU_NAME + '_results_v' + str(VERSION) + '.json'

load_dotenv(find_dotenv())

## Train/ load NLU model

### Snips.ai

In [ ]:
def read_json_file(filename):
    with open(filename) as f:
        data = json.load(f)
    return data

In [ ]:
train_dataset = read_json_file(TRAIN_FILE)
nlu_engine = SnipsNLUEngine(config=CONFIG_EN)
nlu_engine.fit(train_dataset)

### Watson Assistant

In [ ]:
# documentation: https://cloud.ibm.com/apidocs/assistant/assistant-v2?code=python
authenticator = IAMAuthenticator(os.getenv("IBM_ASSISTANT_API_KEY"))
assistant = AssistantV2(
    version='2022-02-02',
    authenticator = authenticator
)
assistant.set_service_url(os.getenv("IBM_WATSON_REGION"))
## TODO: get the correct skill depending on the domain (should be created manually first in IBM's interface)

## Test NLU (works for Watson, LUIS and Snips.ai)

In [ ]:
def unify_keys(json_res, nlu='snips'):
    for rank in json_res['intent_ranking']:
        if nlu=='snips':
            rank['name'] = rank.pop('intentName')
            rank['confidence'] = rank.pop('probability')
        elif nlu=='watson':
            rank['name'] = rank.pop('intent')
        else:
            pass
    return json_res

In [ ]:
def add_is_correct(json_res):
    if json_res['intent_ranking'][0]['name'] == intent:
        json_res['is_correct'] = True
    else:
        json_res['is_correct'] = False
    return json_res   

In [ ]:
def get_nlu_response(utterance, assistant=None, nlu='snips'):
    if nlu=='watson':
        response = assistant.message_stateless(
            assistant_id=os.getenv("IBM_ASSISTANT_ID"),
                input={
                    'message_type': 'text',
                    'text': utterance,
                    'options' : {'alternate_intents': True}
                }
            ).get_result()
    elif nlu=='snips':
        response = nlu_engine.get_intents(utterance)
    elif nlu=='luis':
        appId = os.getenv("LUIS_APP_ID")
        prediction_key = os.getenv("LUIS_PREDICTION_SUBSCRIPTION_KEY")
        prediction_endpoint = os.getenv("LUIS_PREDICTION_ENDPOINT")
        # The URL parameters to use in this REST call.
        headers = {}
        params ={
            'query': utterance,
            'timezoneOffset': '0',
            'verbose': 'true',
            'show-all-intents': 'true',
            'spellCheck': 'false',
            'staging': 'false',
            'subscription-key': prediction_key
        }

        # Make the REST call.
        response = requests.get(f'{prediction_endpoint}luis/prediction/v3.0/apps/{appId}/slots/production/predict', headers=headers, params=params)
        response = response.json()
    else:
        pass
    
    return response

In [ ]:
def sleep_if_luis(i, nlu='luis'): 
    if i % 5 == 4 and nlu=='luis': # not 0 because indecies start at 0
        time.sleep(1.5) # to keep a rate of 5TC per second

In [ ]:
file = open(TEST_FILE)
reader = csv.reader(file, delimiter=',')

In [ ]:
results = []
for i, row in enumerate(reader):
    intent = row[1]
    utterance = row[0]
    
    if i==0 or utterance=='': # skip header and empty utterances
        continue 
        
    sleep_if_luis(i, nlu=NLU_NAME)
    
    if NLU_NAME == 'watson':
        response = get_nlu_response(utterance, assistant=assistant, nlu=NLU_NAME)
        intent_ranking = response['output']['intents']
    elif NLU_NAME == 'snips':
        intent_ranking = get_nlu_response(utterance, nlu=NLU_NAME)
        intent_ranking = [r for r in intent_ranking if r['intentName'] != None]
    elif NLU_NAME == 'luis':
        response = get_nlu_response(utterance, nlu=NLU_NAME)
        try:
            intent_ranking = response['prediction']['intents']
            intent_ranking = [{'name' : n, 'confidence': s['score']} for n,s in intent_ranking.items()]
        except:
            print(response)
            print("Trying to recover ...")
            time.sleep(1)
            response = get_nlu_response(utterance, nlu=NLU_NAME)
            try:
                intent_ranking = response['prediction']['intents']
                intent_ranking = [{'name' : n, 'confidence': s['score']} for n,s in intent_ranking.items()]
            except:
                print("Failed to recover. Adding an empty list to intent_ranking")
                intent_ranking = []
        
    json_res = { 'text': utterance,
                 'correct_intent' : intent,
                 'intent_ranking' : intent_ranking
                }
    
    try:
        json_res = unify_keys(json_res, nlu=NLU_NAME)
        json_res = add_is_correct(json_res)
        
        results.append(json_res)
    except:
        # ignored predictions
        print("this is ignored", json_res)

In [ ]:
with open(OUTPUT_FILE, 'w') as f:
    json.dump(results, f, indent=2)